In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

In [3]:
image_dir = r"D:\train"
img_size = (48, 48)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    image_dir,
    target_size=img_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    subset='training'
)

val_gen = datagen.flow_from_directory(
    image_dir,
    target_size=img_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation'
)


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [4]:
model = Sequential([
    Input(shape=(48, 48, 1)),  # Use Input layer for defining input shape
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])


In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10  # You can increase for better results
)

Epoch 1/10


D:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


718/718 ━━━━━━━━━━━━━━━━━━━━ 67s 89ms/step - accuracy: 0.2519 - loss: 1.8165 - val_accuracy: 0.4036 - val_loss: 1.5234
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 7551s 11s/step - accuracy: 0.4119 - loss: 1.5201 - val_accuracy: 0.4691 - val_loss: 1.3794
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 62s 86ms/step - accuracy: 0.4698 - loss: 1.3769 - val_accuracy: 0.4924 - val_loss: 1.3392
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 60s 84ms/step - accuracy: 0.5225 - loss: 1.2685 - val_accuracy: 0.5158 - val_loss: 1.2663
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 680s 949ms/step - accuracy: 0.5477 - loss: 1.2060 - val_accuracy: 0.5368 - val_loss: 1.2364
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 65s 91ms/step - accuracy: 0.5682 - loss: 1.1421 - val_accuracy: 0.5144 - val_loss: 1.2607
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 64s 89ms/step - accuracy: 0.5822 - loss: 1.1075 - val_accuracy: 0.5485 - val_loss: 1.2137
Epoch 8/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 63s 87ms/step - accuracy: 0.6084 - loss: 1.0604 - val_accu

In [9]:
model.save('model.keras')
print("Model saved as emotion_model.h5 ✅")


Model saved as emotion_model.h5 ✅


In [10]:

from tensorflow.keras.models import load_model


In [ ]:
# Load your trained CNN model
model = load_model("model.keras")  # Replace with your model path

# Label map (match the order of your training folders)
emotion_labels = ['angry', 'disgusted','fearful','happy','neutral', 'sad', 'surprised']

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_roi, (48, 48))
        input_img = face_resized.reshape(1, 48, 48, 1) / 255.0

        pred = model.predict(input_img)
        emotion_idx = np.argmax(pred)
        emotion = emotion_labels[emotion_idx]

        # Draw bounding box and emotion label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
emoji_map = {
    0: "😡",  # angry
    1: "😄",  # happy
    2: "😢",  # sad
    3: "😮",  # surprised
    4: "🤢",  # disgusted
    5: "😱",  # fearful
    6: "😐"   # neutral
}


In [ ]:
import cv2
import numpy as np

def webcam_predict():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (48, 48))
            input_img = face_resized.reshape(1, 48, 48, 1) / 255.0

            prediction = model.predict(input_img)
            class_idx = np.argmax(prediction)
            emoji = emoji_map.get(class_idx, "?")

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emoji, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

        cv2.imshow("Real-Time Emotion to Emoji", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
